In [38]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 020c0749-f048-4d9f-91a9-40eeb1f21be3)

In [39]:
#ckecking only on column
df = df.select('value')

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 41, Finished, Available, Finished)

In [40]:
#this is how value column looks like
display(df)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fbdf2d37-d88a-4136-98a3-d95d1e8cd5fa)

In [41]:
#exploding my df using explod function on pyspark it has nested json
from pyspark.sql.functions import explode

df_explode = df.select(explode(df["value"]).alias("json_object"))
display(df_explode)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 43, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ddbee2af-e7a8-47a4-ad90-26d4b2994b7e)

In [42]:
#converting it into list
json_list = df_explode.toJSON().collect()

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 44, Finished, Available, Finished)

In [43]:
#looking into last value of list
print(json_list[-1])

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 45, Finished, Available, Finished)

{"json_object":{"category":"LifeStyle","datePublished":"2024-10-17T22:56:00.0000000Z","description":"Universal Studios Orlando has announced that Epic Universe, its highly-anticipated fourth theme park in Florida, will open sooner than expected - May 2025. Soon people will be able to purchase tickets to see the new theme park,","name":"Epic Universe opening May 2025: When you can buy tickets, book hotel rooms, new lands and attractions","provider":[{"_type":"Organization","image":{"thumbnail":{"contentUrl":"https://www.bing.com/th?id=ODF.pPTO_qfDGMw5U2MDJ6xzpw&pid=news"}},"name":"FOX 35 Orlando on MSN.com"}],"url":"https://www.msn.com/en-us/travel/tripideas/epic-universe-opening-may-2025-when-you-can-buy-tickets-book-hotel-rooms-new-lands-and-attractions/ar-AA1srljb"}}


In [44]:
#converting to dict so it will be easy to query
import json
news_json = json.loads(json_list[-1])
print(news_json)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 46, Finished, Available, Finished)

{'json_object': {'category': 'LifeStyle', 'datePublished': '2024-10-17T22:56:00.0000000Z', 'description': 'Universal Studios Orlando has announced that Epic Universe, its highly-anticipated fourth theme park in Florida, will open sooner than expected - May 2025. Soon people will be able to purchase tickets to see the new theme park,', 'name': 'Epic Universe opening May 2025: When you can buy tickets, book hotel rooms, new lands and attractions', 'provider': [{'_type': 'Organization', 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=ODF.pPTO_qfDGMw5U2MDJ6xzpw&pid=news'}}, 'name': 'FOX 35 Orlando on MSN.com'}], 'url': 'https://www.msn.com/en-us/travel/tripideas/epic-universe-opening-may-2025-when-you-can-buy-tickets-book-hotel-rooms-new-lands-and-attractions/ar-AA1srljb'}}


In [45]:
#testing with one value to see how to query my dict
print(news_json['json_object']['name'])
print(news_json['json_object']['description'])
print(news_json['json_object']['category'])
print(news_json['json_object']['url'])
print(news_json['json_object']['provider'][0]['image']['thumbnail']['contentUrl'])
print(news_json['json_object']['provider'][0]['name'])
print(news_json['json_object']['datePublished'])

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 47, Finished, Available, Finished)

Epic Universe opening May 2025: When you can buy tickets, book hotel rooms, new lands and attractions
Universal Studios Orlando has announced that Epic Universe, its highly-anticipated fourth theme park in Florida, will open sooner than expected - May 2025. Soon people will be able to purchase tickets to see the new theme park,
LifeStyle
https://www.msn.com/en-us/travel/tripideas/epic-universe-opening-may-2025-when-you-can-buy-tickets-book-hotel-rooms-new-lands-and-attractions/ar-AA1srljb
https://www.bing.com/th?id=ODF.pPTO_qfDGMw5U2MDJ6xzpw&pid=news
FOX 35 Orlando on MSN.com
2024-10-17T22:56:00.0000000Z


In [46]:
## Below are the column I m interested for my project
#description
#title
#category
#image
#url
#provider
#publishedDate

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 48, Finished, Available, Finished)

In [47]:
import json

description = []
title = []
category = []
image = []
url = []
provider = []
publishedDate = []

# Process each JSON object into list
for json_lst in json_list:
    try:
        # Parse JSON list into dict
        news = json.loads(json_lst)

        # Extract info from dict using get method
        title.append(news['json_object'].get('name', 'N/A'))  # Default to 'N/A' if 'name' key is missing
        description.append(news['json_object'].get('description', 'N/A'))
        category.append(news['json_object'].get('category', 'N/A'))
        url.append(news['json_object'].get('url', 'N/A'))

        # Check if 'provider' exists and contains the required subkeys
        provider_info = news['json_object'].get('provider', [])
        if provider_info and isinstance(provider_info, list) and 'image' in provider_info[0]:
            image.append(provider_info[0].get('image', {}).get('thumbnail', {}).get('contentUrl', 'N/A'))
        else:
            image.append('N/A')

        provider.append(provider_info[0].get('name', 'N/A') if provider_info else 'N/A')
        publishedDate.append(news['json_object'].get('datePublished', 'N/A'))

    except Exception as e:
        print(f'Error processing JSON object: {e}')


StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 49, Finished, Available, Finished)

In [48]:
from pyspark.sql.types import StructType, StringType, StructField

#combine a list
data = list(zip(description, title, category, image, url, provider, publishedDate))

#define scheme
schema = StructType([
    StructField('title', StringType(), True),
    StructField('description', StringType(), True),
    StructField('category', StringType(), True),
    StructField('image', StringType(), True),
    StructField('url', StringType(), True),
    StructField('provider', StringType(), True),
    StructField('publishedDate', StringType(), True)
])

#created dataframe
df_final = spark.createDataFrame(data, schema=schema)
display(df_final)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 50, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9364a3bd-99e0-45b1-83ec-c503edcc0d21)

In [51]:
# converting string into date
from pyspark.sql.functions import to_date, date_format

df_final1 = df_final.withColumn('publishedDate', date_format(to_date('publishedDate'),'dd-MMM-yyyy'))

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 53, Finished, Available, Finished)

In [52]:
display(df_final1)

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 54, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1e684c40-ce97-4bf6-95f1-c54c433a7bc1)

In [53]:
# loading data into final destination
df_final1.write.format('delta').mode('append').saveAsTable('bing_link_db.tbl_latest_news')

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 55, Finished, Available, Finished)

In [ ]:
# Using sql type 1 condition to update my existing table
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_link_db.tbl_latest_news'
    df_final1.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print('Table Already Exist')
    df_final1.createOrReplaceTempView('vw_df_cleaned_final')

    spark.sql(f""" Merge into {table_name} target_table
                   USING vw_df_cleaned_final source_view
                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.publishedDate <> target_table.publishedDate OR

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
                
                """)

In [54]:
description = []
title = []
category = []
image = []
url = []
provider = []
publishedDate = []
%%sql

SELECT count(*) from bing_link_db.tbl_latest_news

StatementMeta(, c8d1c0e5-cca0-4714-b512-20d9c8cdf90e, 56, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>